一共选取六大类因子中的106个特征值+10个风险因子，进行回归预测
回归目标值为：股票涨跌幅
滚动预测：前12期预测下一期（月频数据）
        前48期预测下一期（周频数据）

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import daolib.dao as dao
import daolib.dso as dso
import util.sectool as sectool
import util.operatetool as optool
import matplotlib.pyplot as plt
import pickle
import time
from tqdm import tqdm
%matplotlib inline

## 读取数据

In [ ]:
#直接从文件夹读取分类构造好的数据
week_alpha_factor_series=pickle.load(open("./factor_test_data/LightGBM_allfactor/week_alpha_factor_series","rb+"))  #周频数据
month_alpha_factor_series=pickle.load(open("./factor_test_data/LightGBM_allfactor/month_alpha_factor_series","rb+")) #月频数据

## 概率预测

### 单个模型评估

In [ ]:
Wscore_lasso,Wscore_ENet,Wscore_KRR ,Wscore_GBoost,Wscore_xgb,Wscore_lgb =  rmse_siglemodel(week_alpha_factor_series,12,week_list) #周频数据

In [ ]:
#RMSE画图分析（看rmse平均值）
rmse_list=[lasso_rmse,ENet_rmse,xgb_rmse,lgb_rmse,GBoost_rmse]
label_list=['lasso_rmse','ENet_rmse','xgb_rmse','lgb_rmse','GBoost_rmse']
rmse_show(rmse_list,label_list)

In [ ]:
# 最终模型的预测结果
lasso_rmse,lasso_predict_value=rmsle(lasso,month_alpha_factor_series,12,month_list)
ENet_rmse,ENet_predict_value=rmsle(ENet,month_alpha_factor_series,12,month_list)
lgb_rmse,lgb_predict_value=rmsle(model_lgb,month_alpha_factor_series,12,month_list)
xgb_rmse,xgb_predict_value=rmsle(model_xgb,month_alpha_factor_series,12,month_list)
GBoost_rmse,GBoost_predict_value=rmsle(GBoost,month_alpha_factor_series,12,month_list)

### 集成模型预测

#### 简单平均模型融合，（选择最小rmse的三个模型），计算融合模型的rmse

In [ ]:
averaged_models = AveragingModels(models = (ENet,model_lgb,lasso,model_lgb))
averaged_score1 = rmsle_cv(averaged_models,month_alpha_factor_series,12,month_list)
averaged_score2,averaged_predict_value=rmsle(averaged_models,month_alpha_factor_series,12,month_list)

#### Meta-model模型融合

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet,model_lgb, model_xgb),meta_model = lasso)
stacked_averaged_score1 = rmsle_cv(stacked_averaged_models,month_alpha_factor_series,12,month_list)
stacked_averaged_score2,stacked_averaged_predict_value=rmsle(stacked_averaged_models,month_alpha_factor_series,12,month_list)

#### 数据融合

In [ ]:
#最终的数据融合（分配不同权重）
# ensemble1 = GBoost_predict_value*0.7+ xgb_predict_value*0.15 +lgb_predict_value*0.15
# ensemble2 = GBoost_predict_value*0.8+ xgb_predict_value*0.1 +lgb_predict_value*0.1
ensemble3 = GBoost_predict_value*0.6+ xgb_predict_value*0.2 +lgb_predict_value*0.2

### 因子分析

#### 因子保存

In [ ]:
import pickle
stacked_averaged_predict_value.to_pickle("./factor_test_data/LightGBM_allfactor2/stacked_averaged_predict_value")

#### 因子测试

In [ ]:
#因子测试1
averaged_predict_value_obj=factor_analyse('test1',averaged_predict_value)
#因子测试2
average_samew_pre,average_samew_unpre,average_unw_pre,average_unw_unpre=factor_stock_choose(averaged_predict_value,100)

## 函数部分

### 回归模型（lasso，ENet，KRR，GBoost，model_xgb，model_lgb)

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,max_depth=4, max_features='sqrt',)

RF=RandomForestRegressor(max_depth=5)

model_xgb = xgb.XGBRegressor(learning_rate=0.05, max_depth=5, )

model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=50,learning_rate=0.05)

### 单个模型分数评估

In [ ]:
#模型分数评估（采用交叉验证三折评估，计算RMSE）
n_folds=3
def rmsle_cv(model,data,M,date_list):
    RMSE=[]
    train,test=data_input_reg(data,date_list) 
    for m in range(len(date_list)-M-4):
        X_train, X_test, Y_train, Y_test,Xx_test= splitdata(data,train,test,m+M-1,m,m+M-1,m+M)
        kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train)
        rmse= np.sqrt(-cross_val_score(model, X_train, Y_train, scoring="neg_mean_squared_error", cv = kf))
        RMSE.append(rmse)
    mean_rmse=np.array(RMSE).mean()
    return mean_rmse

def rmse_siglemodel(data,loop,date_list):    #单个回归模型评估（三折评估计算rmse）
    regscore_lasso= rmsle_cv(lasso,data,loop,date_list)
    regscore_ENet = rmsle_cv(ENet,data,loop,date_list)
    regscore_KRR = rmsle_cv(KRR,data,loop,date_list)
    regscore_GBoost = rmsle_cv(GBoost,data,loop,date_list)
    regscore_xgb = rmsle_cv(model_xgb,data,loop,date_list)
    regscore_lgb = rmsle_cv(model_lgb,data,loop,date_list)
    return   regscore_lasso,regscore_ENet,regscore_KRR ,regscore_GBoost,regscore_xgb,regscore_lgb   

### 最终预测结果

In [ ]:
def rmsle(model,data,loop,date_list):
    RMSE=[]
    predict_value=[]
    train,test=data_input_reg(data,date_list) 
    for m in range(len(date_list)-loop-4):
        X_train, X_test, Y_train, Y_test,Xx_test= splitdata(data,train,test,m+loop-1,m,m+loop-1,m+loop)
        model.fit(X_train, Y_train)
        train_pred = model.predict(X_train)
        y_pred =model.predict(X_test)
        y_pred=pd.DataFrame(y_pred)
        y_pred['stock']=Xx_test[:,-2]
        y_pred['date']=Xx_test[:,-1] 
        rmse=np.sqrt(mean_squared_error(Y_train, train_pred))
        RMSE.append(rmse)
        predict_value.append(y_pred)
    factor=changeindex1(predict_value,loop,date_list)
    return RMSE,factor

### 准确率分析

In [ ]:
#RMSE画图分析（看平均值）
def rmse_show(rmse_list,label_list):
    for i in rmse_list:
        plt.plot(i,"x-",label=label_list[rmse_list.index(i)])
    plt.legend() 
    for i in rmse_list:
        print(label_list[rmse_list.index(i)],"平均RMSE为:",np.array(i).mean())

### 模型融合

#### 简单平均模型融合

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        for model in self.models_:
            model.fit(X, y)
        return self
    def predict(self, X):
        predictions = np.column_stack([model.predict(X) for model in self.models_])
        return np.mean(predictions, axis=1)  

#### Meta-model Stacking

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds

    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

### 数据构造

In [ ]:
#标签值-回归
def rise_fall_reg(trade_date_m_series):
    zz_df = dao.get_index_component_data('ZZ')
    stock_price_df = dao.get_security_info('stock_price_info')[trade_date_m_series]
    stock_close_df = stock_price_df.xs('close',level=1)[trade_date_m_series]
    trade_status_df = stock_price_df.xs('trade_status', level=1)[trade_date_m_series]
    pause_df = trade_status_df.copy()
    pause_df[pause_df == 1] = np.nan
    pause_df[pause_df==0] = 1

    stock_chg_df = stock_close_df.pct_change(axis=1)
    stock_return_df = stock_chg_df * pause_df  * zz_df
    stock_return_df = stock_return_df.shift(-1, axis=1)
    return stock_return_df

#划分测试集和训练集
def concat_data(data,date_list,is_rise_df):
    factor_class_series = data.map(lambda x: x.loc[:,date_list[0]:date_list[-1]])
    data_df = pd.DataFrame()
    factor_name_list = factor_class_series.index.tolist()
    data_dict = {}
    for trade_date in tqdm(date_list[:]):
        data_section_series = factor_class_series.map(lambda x: x[trade_date] if trade_date in x.columns else None)
        data_section_df = pd.DataFrame(data_section_series.to_dict())
        data_section_df =data_section_df.reindex(columns=factor_name_list)
        data_section_df['rise_fall'] = is_rise_df[trade_date]
        data_dict[trade_date] = data_section_df
        data_section_df['date'] = trade_date
    return data_dict
    
def data_train_test(data_pct):
    data_pct_test=data_pct      #包含0，-1,1的三种分类的全部数据预测集
    data_pct_test=data_pct[data_pct['trade_status']==0]        #选择正常股票状态的数据
    data_pct_test=data_pct_test.dropna()    #删除空值（回归空值太多）
    data_pct_train=data_pct     #不包含0的训练集
    data_pct_train=data_pct_train[data_pct_train['trade_status']==0]       #选择正常股票状态的数据
    data_pct_train=data_pct_train.dropna()
    return data_pct_train,data_pct_test

def data_input_reg(data,date_list):
    is_rise_df=rise_fall_reg(date_list)    
    data_dict=concat_data(data,date_list,is_rise_df)
    data_df = pd.concat([data_dict[frame] for frame in data_dict.keys()])
    train1,test1=data_train_test(data_df) 
    return train1,test1

# 数据标准化，可以处理空值
def standard(X_train,X_test):
    X_train_scaled =1.0 * (X_train - X_train.mean()) / X_train.std()  # 数据标准化
    X_test_scaled =1.0 * (X_test - X_test.mean()) / X_test.std()  # 数据标准化
    return  X_train_scaled,X_test_scaled 

def countsum(data):
    a=data.reset_index()
    a.rename(columns=lambda x:x.replace('index','stock'), inplace=True) 
    resultdata=(a['stock'].groupby(a['date'])).describe()
    resultdata['sum']=resultdata['count'].cumsum()
    return resultdata,a

#两分类划分，划分训练集data，测试集alldata（训练集的类别只有（0,1），测试集包含所有类别（0,1，-1））
def splitdata(data,train,test,i,j,x,y):
    resultdata,a=countsum(train)
    resultalldata,b=countsum(test)
    i=resultdata['sum'][i]
    j=resultdata['sum'][j]
    x=resultalldata['sum'][x]
    y=resultalldata['sum'][y]

    newname=data.index.tolist()    
    newname.append('stock')
    newname.append('date')

    X_train=np.array(a[newname][j:i])
    Y_train=np.array(a['rise_fall'][j:i])
    #第x个月，测试集
    X_test=np.array(b[newname][x:y])
    Y_test=np.array(b['rise_fall'][x:y])
    X_train_scaled,X_test_scaled=X_train[:,:-3],X_test[:,:-3]
    return X_train_scaled,X_test_scaled,Y_train,Y_test,X_test

### 因子数据合成

In [ ]:
import itertools

def change(data,n,m,M,date_list):
    date=date_list[M:-4]
    factor_df=pd.DataFrame(columns=date)
    factor_df['stock']=list(month_alpha_factor_series[100].index)
#     factor_df['stock']=data['stock']

    for i,t in itertools.zip_longest(data,date):
        temp=factor_df[['stock']]
        temp[t]=np.nan
        u=i.iloc[:,[n,m]]
        u.columns=[t,'stock']
        factor_Crash=pd.concat([u,temp],join='inner',ignore_index=True)
        factor_Crash.sort_values(t,inplace=True)
        factor_Crash.drop_duplicates(['stock'],inplace=True)
        factor_Crash.sort_values('stock',inplace=True)
        factor_Crash.reset_index(inplace=True)
        factor_df[t]= factor_Crash[t]
    factorF_df=factor_df.set_index(['stock'])
    return factorF_df

def changeindex1(data,M,date_list):
    factor_df=change(data,0,1,M,date_list)
    return  factor_df

### 因子显著度T检验

In [ ]:
import util.factortool as ftool
def factor_test_T(factor_list,factor_name): 
    risk_test=pd.DataFrame()
    for i ,n in itertools.zip_longest(factor_list,factor_name):
        risk_test[n]=ftool.factor_risk_test_tvalue(i)
    return  risk_test

### 因子测试1

In [ ]:
import alphafactors.factorprepro_class as fp
import alphafactors.factoranalyse as fa

#因子处理(分成两种方向)
def factor_analyse(name,factor):  # 0-positive , 1-negetive
    factor_prepro_obj = fp.FactorPrePro(factor_name=name, factor_data_df=factor, universe='ZZ', neutral_list=None)
    factor_prepro_obj.run_process(start_date=max(factor.columns[0], dt.datetime(2007,1,5)), end_date=factor.columns[-1])
    df = factor_prepro_obj.factor_pro_df
    factor_test_obj = fa.FactorAnalyse(factor_name=name, factor_data_df=df, factor_dr=0)   # 0-positive , 1-negetive
    factor_test_obj.run_analyse_new(start_date=dt.datetime(2009,1,23), universe='ZZ')
    return factor_test_obj

#因子测试画图显示
def show1(factor_test_obj):
    factor_test_obj.net_value_df.iloc[:,-3:].plot(figsize=(20,10))
def show2(factor_test_obj):
    factor_test_obj.factor_perform_df
    return  factor_test_obj.factor_perform_df
def show3(factor_test_obj):
    factor_test_obj.factor_para_df
    return  factor_test_obj.factor_para_df
def show4(factor_test_obj):
    factor_test_obj.port_perform_df
    return     factor_test_obj.port_perform_df
def show5(factor_test_obj):
    factor_test_obj.port_perform_df['annual_return'].plot(kind='bar')
    return factor_test_obj.port_perform_df['annual_return'].plot(kind='bar')   
def show6(factor_test_obj):
    factor_test_obj.factor_index_df['IC值'].plot(kind='bar', figsize=(20,10), color='blue')
    return  factor_test_obj.factor_index_df['IC值'].plot(kind='bar', figsize=(20,10), color='blue')

### 因子测试2

In [ ]:
import util.evalstat as evl
                                     
def factor_test_pre(factor):         #因子中性化预处理
    factor_prepro_obj = fp.FactorPrePro(factor_name='factor_test', factor_data_df=factor, universe='ZZ', neutral_list=None)
    factor_prepro_obj.run_process(start_date=max(factor.columns[0], dt.datetime(2007,1,5)), end_date=factor.columns[-1])
    df = factor_prepro_obj.factor_pro_df
    return df

def factor_test(stock_weighted_series):
    perform_obj=evl.PortPerform(port_series=stock_weighted_series,ret_type='open',fee=0.0035)
    perform_obj.run()
    return perform_obj

def show01(perform_obj):
    perform_obj.net_value_plot()        
def show02(perform_obj):
    perform_obj.get_strategy_perform() 
    return perform_obj.get_strategy_perform()      
def show03(perform_obj):
    perform_obj.get_avg_turnover()  
    return perform_obj.get_avg_turnover()      
def show04(perform_obj):
    perform_obj.get_annual_perform()  
    return  perform_obj.get_annual_perform()  

### 投资组合构建（2种：一种等权，一种不等权）

In [ ]:
def stock_choice(data,num):                 #直接挑选概率值前100支股票，等权
    stock_series= pd.Series()
    for i in data.columns:
        stock_series.loc[i]=pd.Series(index=[data[i].sort_values(ascending=False).head(num).index],data=1/num)
    stock_choice_obj=factor_test(stock_series)
    return stock_choice_obj

def stock_bench_ind(data,num):   #行业中性，基准权重后挑选100支股票
    stock_series= pd.Series()
    for i in data.columns:
       set_date=i
       stock_series[i]=get_industry_stock(data[i], set_date, stock_num=num)
    stock_choice_obj =factor_test(stock_series)
    return stock_choice_obj

def factor_stock_choose(factor,num):
    factor_obj=factor_test_pre(factor)   #做因子预处理
    #等权选num支
    samew_pre=stock_choice(factor_obj,num)
    samew_unpre=stock_choice(factor,num)   #不做因子预处理，等权直接选100支
    #不等权选num支
    unw_pre=stock_bench_ind(factor_obj,num)
    unw_unpre=stock_bench_ind(factor,num)
    return  samew_pre,samew_unpre,unw_pre,unw_unpre

### 基准行业权重

In [ ]:
stock_industry_df = dao.get_security_info('stock_industry_CS')
stock_industry_list = dso.get_industry_classify('CS')
stock_pool_df = dao.get_index_component_data('DEF')

def get_bench_ind_weight(set_date, bench_code='ZZ500'):
    industry_series = optool.get_series_from_df(data_df=stock_industry_df, set_date=set_date, axis=1).dropna()
    group_data = industry_series.index.groupby(industry_series)
    # 基准行业权重
    bench_component_df = dao.get_index_component_data(bench_code)
    bench_series = pd.Series(bench_component_df[set_date].set_index('code')['weight'])
    bench_series = bench_series / bench_series.sum()
    bench_ind_weight_series = pd.Series(index=stock_industry_list)
    for industry_name in stock_industry_list:
        ind_stock_list = group_data[industry_name]
        temp_series = bench_series.copy()
        bench_ind_weight_series.loc[industry_name] = temp_series.reindex(ind_stock_list).sum()
    bench_ind_weight_series.fillna(0, inplace=True)
    return bench_ind_weight_series

def get_industry_stock(stock_factor_series, set_date, stock_num=100):
    industry_series = optool.get_series_from_df(data_df=stock_industry_df, set_date=set_date, axis=1).dropna()
    stock_series = stock_pool_df[set_date]
    stock_list = industry_series.index.intersection(stock_series.dropna().index).tolist()
    industry_series = industry_series.loc[stock_list]

    group_data = industry_series.index.groupby(industry_series)
    # 基准行业权重
    bench_ind_weight_series = get_bench_ind_weight(set_date=set_date)
    bench_ind_num_series = round(bench_ind_weight_series * stock_num).astype(int)
    port_series = pd.Series()
    # 得到行业中性组合
    for industry_name in stock_industry_list[:]:
        if bench_ind_weight_series[industry_name] <= 0.0:
            continue
        ind_stock_list = group_data[industry_name]
        ind_stock_series = pd.Series(stock_factor_series.loc[ind_stock_list]).reindex(ind_stock_list).sort_values(
            ascending=False)

        ind_stock_num = bench_ind_num_series[industry_name]
        if ind_stock_num < 1:
            ind_stock_num += 1

        if ind_stock_series.shape[0] >= ind_stock_num:
            xx = ind_stock_series.head(ind_stock_num)
            temp_series = xx / xx.sum() * bench_ind_weight_series[industry_name]
        else:
            temp_series = ind_stock_series / ind_stock_series.sum() * bench_ind_weight_series[industry_name]

            temp_series = pd.Series(index=ind_stock_series.index[:ind_stock_num], data=1.0 / ind_stock_num) * \
                          bench_ind_weight_series[industry_name]
        port_series = port_series.append(temp_series)
    return port_series